# Análise de logs de fuzzing

Coloque os arquivos `.jsonl` em `../sample_logs/` e execute as células abaixo.

In [ ]:
import pandas as pd
import json
from glob import glob

files = glob('../sample_logs/*.jsonl')
files

In [ ]:
# Carregar todos os logs em um único DataFrame
dfs = []
for f in files:
    try:
        d = pd.read_json(f, lines=True)
        d['source'] = f
        dfs.append(d)
    except Exception as e:
        print('erro ao ler', f, e)

if dfs:
    df = pd.concat(dfs, ignore_index=True)
    display(df.head())
else:
    print('Nenhum arquivo encontrado em ../sample_logs/')

In [ ]:
# Filtrar potenciais falhas (5xx ou erros de request)
if 'status' in df.columns:
    failures = df[(df['status'] >= 500) | (df.get('ok') == False)]
    print('Falhas encontradas:', len(failures))
    if not failures.empty:
        failures['signature'] = failures['status'].astype(str) + failures['body'].astype(str).str.slice(0,200)
        print('Falhas únicas (aprox):', failures['signature'].nunique())
else:
    print('Coluna status não encontrada nos logs')